Testing this hypothesis: Car price and customer satisfaction will be directly correlated, with consumer ratings rising as prices rise up to a certain point. After this certain point, the ratings plateau or fall.

Attempting to test this correlation.

Import needed libraries, and import dataset into a pandas dataframe

In [112]:
import pandas as pd
import math
import numpy as np

df = pd.read_csv("cars_raw.csv")

df.head()

,Year,Make,Model,Used/New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock#,Mileage
0,2019,Toyota,Sienna SE,Used,"$39,998",4.6,45,Dealer,CarMax Murrieta - Now offering Curbside Pickup...,3.3,...,Black,Front-wheel Drive,19,27,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,5TDXZ3DC2KS015402,22998646,29403
1,2018,Ford,F-150 Lariat,Used,"$49,985",4.8,817,Dealer,Giant Chevrolet,4.8,...,Black,Four-wheel Drive,19,24,Gasoline,10-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,1FTEW1EG2JFD44217,22418A,32929
2,2017,RAM,1500 Laramie,Used,"$41,860",4.7,495,Dealer,Gill Auto Group Madera,4.6,...,Black,Four-wheel Drive,15,21,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6RR7VT5HS842283,NG277871G,23173
3,2021,Honda,Accord Sport SE,Used,"$28,500",5.0,36,Dealer,AutoSavvy Las Vegas,4.6,...,–,Front-wheel Drive,29,35,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,1HGCV1F49MA038035,54237,10598
4,2020,Lexus,RX 350,Used,"$49,000",4.8,76,Dealer,Lexus of Henderson,4.8,...,Birch,Front-wheel Drive,20,27,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,2T2AZMAA8LC156270,HDT4181A,28137


Since we're just working with price and consumer rating, drop all other columns

In [113]:
df_temp= df[['Price', 'ConsumerRating']]
df= df_temp.sort_values(by= 'Price', ascending= True, kind= "mergesort")
df

,Price,ConsumerRating
8599,"$10,499",4.4
8229,"$10,775",4.4
2039,"$10,795",4.7
5313,"$10,795",4.7
6570,"$10,799",4.4
...,...,...
58,Not Priced,4.9
2810,Not Priced,4.9
4226,Not Priced,4.6
6065,Not Priced,4.9


Clean price data by removing all the rows that aren't priced

In [115]:
df= df.drop(df[df.Price== "Not Priced"].index)
df

,Price,ConsumerRating
8599,"$10,499",4.4
8229,"$10,775",4.4
2039,"$10,795",4.7
5313,"$10,795",4.7
6570,"$10,799",4.4
...,...,...
2732,"$99,980",5.0
5261,"$99,980",5.0
8655,"$99,985",4.8
8233,"$99,997",4.7


Transform all $ currency values as a float by removing $ and comma signs

In [116]:
df[df.columns[0:]] = df[df.columns[0:]].replace('[\$,]', '', regex=True).astype(float)
df

,Price,ConsumerRating
8599,10499.0,4.4
8229,10775.0,4.4
2039,10795.0,4.7
5313,10795.0,4.7
6570,10799.0,4.4
...,...,...
2732,99980.0,5.0
5261,99980.0,5.0
8655,99985.0,4.8
8233,99997.0,4.7


Categorize rows into price ranges

In [117]:
df['Price_Range']=pd.cut(df['Price'], bins= [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000], 
      include_lowest=True,
      labels=['$10k', '$20k', '$30k', '$40k', '$50k', '$60k', '$70k', '$80k', '$90k'])
df


,Price,ConsumerRating,Price_Range
8599,10499.0,4.4,$10k
8229,10775.0,4.4,$10k
2039,10795.0,4.7,$10k
5313,10795.0,4.7,$10k
6570,10799.0,4.4,$10k
...,...,...,...
2732,99980.0,5.0,$90k
5261,99980.0,5.0,$90k
8655,99985.0,4.8,$90k
8233,99997.0,4.7,$90k


Import modules needed for visualization

In [89]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
get_ipython().run_line_magic('matplotlib', 'inline')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import numpy as np

Visualize distribution of each price range

In [96]:
dist = df['Price_Range'].value_counts()
trace = go.Pie(values=(np.array(dist)),labels=dist.index)
layout = go.Layout(title='Distribution by Car Price Range')
data = [trace]
fig = go.Figure(trace,layout)
fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

Now lets use a scatterplot to see if there is any correlation between price and customer satisfaction!

In [111]:
fig = pfig = px.scatter(df, x='Price_Range', y='ConsumerRating',
                       labels={
                           "Price_Range": "Price Range",
                           "ConsumerRating":"Consumer Rating"
                       },
                       title= "Price and Customer Satisfaction")
fig.update_traces(marker_color="orange",marker_line_color='black',
                  marker_line_width=1.5)
#fig.update_layout(title_text='Price and Satisfaction')
fig.show()

Looks like our hypothesis was wrong. Did not finish- not sure why $90k range is out of order on the x axis.